In [1]:
ON_COLAB = False

if ON_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')

In [9]:
# torch
!pip install torch
import torch
from torch.nn import Conv2d, MaxPool2d
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import sampler
!pip install torchvision
from torchvision import datasets, transforms
from torchvision.utils import save_image, make_grid
# other
import sys
sys.path.append('drive/Othercomputers/Il mio laptop/Deep-Pose-Estimation/baseline/')
import params2
!pip install matplotlib
import matplotlib.pyplot as plt
import numpy as np
import os 
from pathlib import Path
!pip install pandas
import pandas as pd
!pip install imageio
import imageio
from csv import writer
from csv import reader
from PIL import Image
!pip install trimesh
import trimesh
!pip install pyrender
import pyrender
if ON_COLAB:
    from tqdm.notebook import tqdm
else:
    !pip install tqdm
    from tqdm import tqdm
'''!pip install wandb
import wandb
!wandb login --relogin'''
# api_key = f2c3bd208d418cf11dcc2c2cbf5e4bc0fc104421

# set the seed for reproducibility
rng_seed = 90
torch.manual_seed(rng_seed)


[notice] A new release of pip available: 22.1.2 -> 22.2.2
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip available: 22.1.2 -> 22.2.2
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip available: 22.1.2 -> 22.2.2
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip available: 22.1.2 -> 22.2.2
[notice] To update, run: python.exe -m pip install --upgrade pip
     ---------------------------------------- 3.4/3.4 MB 15.3 MB/s eta 0:00:00

[notice] A new release of pip available: 22.1.2 -> 22.2.2
[notice] To update, run: python.exe -m pip install --upgrade pip
     -------------------------------------- 660.9/660.9 kB 8.3 MB/s eta 0:00:00

[notice] A new release of pip available: 22.1.2 -> 22.2.2
[notice] To update, run: python.exe -m pip install --upgrade pip
  Using cached pyrender-0.1.45-py3-none-any.whl (1.2 MB)
  Using cached PyOpenGL-3.1.0.zip (2.2 M

ModuleNotFoundError: No module named 'tqdm'

In [ ]:
par = params.Parameters()
print(par.epochs)
print(par.resume)
print(par.run_id)
print(par.batch_size)
print(par.learning_rate)


In [ ]:
os.environ["PYOPENGL_PLATFORM"] = "egl" #opengl seems to only work with TPU
!PYOPENGL_PLATFORM=egl python -c "from OpenGL import EGL"
print(os.environ['PYOPENGL_PLATFORM']) 

import OpenGL.GL as gl
print(gl.glGetString(gl.GL_VERSION))
print(gl.glGetString(gl.GL_VENDOR)) 

In [ ]:
#txt_path = par.txt_path
txt_path = 'drive/Othercomputers/Il mio laptop/Deep-Pose-Estimation/baseline/poses_equatoriale_txt'

def get_nth_line(fobj, n):
    for i in range(n):
        next(fobj)
    return next(fobj)

columns = ['id', 'x', 'y', 'z', 'roll', 'pitch', 'yaw']

with open('drive/Othercomputers/Il mio laptop/Deep-Pose-Estimation/baseline/poses_equatoriale.csv', 'w') as f_object:
    writer_object = writer(f_object)
    writer_object.writerow(columns)
    for f in Path(txt_path).iterdir():
        id = int(str(f).split('/')[-1].split('.txt')[0]) 
        file = open(f)
        line4 = get_nth_line(file, 4).split('\n')[0]
        line_splitted = line4.split(' ')
        writer_object.writerow([id] + line_splitted)
    f_object.close()

In [1]:
class PoseDataset(Dataset):

    def __init__(self, csv_file, root_dir):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        poses = pd.read_csv(csv_file)
        self.poses = poses#.sort_values(by=["id"])
        self.root_dir = root_dir
  

    def __len__(self):
        return len(self.poses)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_path = os.path.join(self.root_dir,
                                str(self.poses.iloc[idx, 0]) + '-N_.png')
        
        img = imageio.imread(img_path, ignoregamma = True)
        rgb_img = img[:,:,:3]
        img_tensor = torch.tensor(rgb_img)/255.0
        img_tensor = np.transpose(img_tensor, (2, 0, 1))
        
        img_downsampled = F.interpolate(img_tensor, scale_factor = 0.25)
        img_downsampled = np.transpose(img_downsampled, (0, 2, 1))
        img_downsampled = F.interpolate(img_downsampled, scale_factor = 0.25)
        img_downsampled = np.transpose(img_downsampled, (0, 2, 1))

        #pose = self.poses.iloc[idx, 1:]
        pose = np.array([pose])
        pose =  pose.reshape(6)
        return img_downsampled, pose

NameError: name 'Dataset' is not defined

In [ ]:
def print_pose(model_path, pose, img):

    tm = trimesh.load(model_path) 
    mesh = pyrender.Mesh.from_trimesh(tm)
    pose = torch.squeeze(pose, dim=0) 
    print('xyz_rpy', pose.cpu().numpy().shape)
    tx, ty, tz, rx, ry, rz = pose.cpu().numpy()
    print('xyz_rpy', pose)

    r = R.from_euler('xyz',[rx, ry, rz], degrees=True)
    #r = R.from_euler('zyx',[rx, ry, rz], degrees=True)

    Twc = np.eye(4)
    Twc[:3,:3] = r.as_matrix()
    Twc[:3,3] = np.array([tx, ty, tz])

    Twc [0,1] *= -1
    Twc [1,0] *= -1
    Twc [1,2] *= -1
    Twc [2,1] *= -1

    Twc [0,3] *= -1

    print('Twc', Twc)

    im_width, im_height = (1024,1024)
    camK = np.array([[886.81,0.0,512.0],[0.0,886.81,512.0],[0.0,0.0,1.0]])

    camera = pyrender.IntrinsicsCamera(camK[0,0],camK[1,1],
                                        camK[0,2],camK[1,2], zfar = 500.0)

    scene=pyrender.Scene()
    scene.add(camera,pose=Twc)
    scene.add(mesh,pose=np.eye(4))
    r = pyrender.OffscreenRenderer(im_width, im_height)
    color,depth = r.render(scene)

    # Flip y in image space
    color = color[::-1,:,:]
    depth = depth[::-1,:]

    # Flip x in image space
    color = color[:,::-1,:]
    depth = depth[:,::-1]

    # Overlap the two images for comparision
    figure = plt.figure(figsize = (10, 10))
    #plt.imshow(img, alpha = 1.0, cmap = plt.cm.gray)
    plt.imshow(depth, cmap = plt.cm.gray_r)
    print('depth', depth)
    wandb.log({"Test samples overlapping pose": figure})
    plt.show()   

In [ ]:
pose_dataset = PoseDataset(csv_file = 'drive/Othercomputers/Il mio laptop/Deep-Pose-Estimation/baseline/poses_equatoriale.csv',
                           root_dir='drive/Othercomputers/Il mio laptop/Deep-Pose-Estimation/baseline/RGB_images_equatoriale')
print(len(pose_dataset))

In [ ]:
# Create train val test split
n = len(pose_dataset)
n_1 = int(n/10)

train_set, val_set, test_set = torch.utils.data.random_split(
    pose_dataset, [n-(2*n_1), n_1, n_1])

print(len(train_set), len(val_set), len(test_set))

loader_train = DataLoader(train_set, batch_size=par.batch_size, shuffle=par.shuffle, num_workers=2)
loader_val = DataLoader(val_set, batch_size=par.batch_size, shuffle=par.shuffle, num_workers=2)
loader_test = DataLoader(test_set, batch_size=par.batch_size, shuffle=par.shuffle, num_workers=2)

In [ ]:
from scipy.spatial.transform import Rotation as R
print(len(loader_train))
for t, (x, y) in enumerate(tqdm(loader_train, leave=False)):
    transform = transforms.ToPILImage()
    x = torch.squeeze(x, dim=0) 
    image_upsampled = F.interpolate(x, scale_factor = 2)
    image_upsampled = np.transpose(image_upsampled, (0, 2, 1))
    image_upsampled = F.interpolate(image_upsampled, scale_factor = 2)
    image_upsampled = np.transpose(image_upsampled, (0, 2, 1))
    image_upsampled = F.interpolate(image_upsampled, scale_factor = 2)
    image_upsampled = np.transpose(image_upsampled, (0, 2, 1))
    image_upsampled = F.interpolate(image_upsampled, scale_factor = 2)
    image_upsampled = np.transpose(image_upsampled, (0, 2, 1))
    image = transform(image_upsampled)
    #img_ddble = cv2.resize(img, (0, 0), fx=0.5, fy=0.5)
    pose = print_pose(model_path=par.tm_path, pose=y, img=image)